<a href="https://colab.research.google.com/github/Aayush452-cell/Machine-Learning-Models/blob/master/Mnist_image_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keras-tuner

In [ ]:
import tensorflow as tf 
from tensorflow import keras
import numpy as np

In [ ]:
print(tf.__version__)

2.2.0


In [ ]:
fashion_mnist=keras.datasets.fashion_mnist

In [ ]:
fashion_mnist

<module 'tensorflow.keras.datasets.fashion_mnist' from '/usr/local/lib/python3.6/dist-packages/tensorflow/keras/datasets/fashion_mnist/__init__.py'>

In [ ]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

In [ ]:
train_images=train_images/255.0
test_images=test_images/255.0

In [ ]:
train_images[0].shape

(28, 28)

In [ ]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [ ]:
def build_model(hp):
  model=keras.Sequential([
                          keras.layers.Conv2D(
                              filters=hp.Int('conv_1_filter',min_value=32,max_value=128,step=16),
                              kernel_size=hp.Choice('conv_1_kernel',values=[3,5]),
                              activation='relu',
                              input_shape=(28,28,1)
                          ),
                        keras.layers.Conv2D(
                            filters=hp.Int('conv_2_filter',min_value=32,max_value=64,step=16),
                            kernel_size=hp.Choice('conv_2_kernel',values=[3,5]),
                            activation='relu'
                        ),
                        keras.layers.Flatten(),
                        keras.layers.Dense(
                            units=hp.Int('dense_1_units',min_value=32,max_value=128,step=16),
                            activation='relu'
                        ),
                        keras.layers.Dense(10,activation='softmax')
                      ])
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2,1e-3])),
                                      loss='sparse_categorical_crossentropy',
                                      metrics=['accuracy'])
  return model

In [ ]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [ ]:
tuner_search=RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='output',project_name='Mnist Fashion')

INFO:tensorflow:Reloading Oracle from existing project output/Mnist Fashion/oracle.json
INFO:tensorflow:Reloading Tuner from output/Mnist Fashion/tuner0.json


In [ ]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

INFO:tensorflow:Oracle triggered exit


In [ ]:
model=tuner_search.get_best_models(num_models=1)[0]

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 48)        480       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 48)        20784     
_________________________________________________________________
flatten (Flatten)            (None, 27648)             0         
_________________________________________________________________
dense (Dense)                (None, 112)               3096688   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1130      
Total params: 3,119,082
Trainable params: 3,119,082
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(train_images,train_labels,epochs=10,validation_split=0.1,initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 16s 9ms/step - loss: 0.1195 - accuracy: 0.9555 - val_loss: 0.2605 - val_accuracy: 0.9143
Epoch 5/10
1688/1688 [==============================] - 15s 9ms/step - loss: 0.0801 - accuracy: 0.9704 - val_loss: 0.2784 - val_accuracy: 0.9147
Epoch 6/10
1688/1688 [==============================] - 15s 9ms/step - loss: 0.0568 - accuracy: 0.9784 - val_loss: 0.3590 - val_accuracy: 0.9165
Epoch 7/10
1688/1688 [==============================] - 15s 9ms/step - loss: 0.0395 - accuracy: 0.9862 - val_loss: 0.4085 - val_accuracy: 0.9165
Epoch 8/10
1688/1688 [==============================] - 14s 9ms/step - loss: 0.0281 - accuracy: 0.9899 - val_loss: 0.4480 - val_accuracy: 0.9163
Epoch 9/10
1688/1688 [==============================] - 15s 9ms/step - loss: 0.0236 - accuracy: 0.9916 - val_loss: 0.4959 - val_accuracy: 0.9155
Epoch 10/10
1688/1688 [==============================] - 15s 9ms/step - loss: 0.0204 - accuracy: 0.9930 - val_loss: 0.4723 - val_a

In [ ]:
model.predict(test_images)

array([[2.1175382e-14, 5.0572115e-15, 3.7556440e-18, ..., 4.7680299e-08,
        4.0523450e-15, 1.0000000e+00],
       [8.1609350e-09, 1.0459730e-17, 1.0000000e+00, ..., 1.7533754e-22,
        4.0400949e-20, 3.3400714e-18],
       [2.4332691e-24, 1.0000000e+00, 8.5089551e-26, ..., 5.2697289e-24,
        6.3837633e-19, 1.6333840e-30],
       ...,
       [2.3958098e-24, 1.4249887e-23, 1.1256099e-21, ..., 6.0510291e-22,
        1.0000000e+00, 3.2071321e-33],
       [2.6825432e-18, 1.0000000e+00, 8.1735103e-23, ..., 2.4759133e-21,
        1.5981152e-25, 2.8025822e-21],
       [1.0492353e-10, 5.9585622e-12, 3.5618446e-11, ..., 3.2593322e-03,
        2.5854672e-05, 3.8332099e-10]], dtype=float32)